# Leakage Channels 的建模

## 电生理学背景

漏电流通道是一类始终开放的离子通道，其通透性不受膜电位、电压门控、配体结合等机制的调控。在电生理学中，漏电通道主要负责维持神经元的静息膜电位，通常被建模为一个线性电导。

在经典的 HH 模型中，漏电流通道主要用于模拟未明确建模的背景离子流，提供一个稳定的背景电流。

## 建模实现

在 `braincell` 中，漏电通道通过继承 `LeakageChannel` 实现。由于漏电电流无激活或失活门控变量，因此该类通道不需要实现 `compute_derivative` 或 `init_state` 等微分过程，相对比较简单。

我们来看一个具体的实现：`IL`，即最基础的线性漏电通道，其数学表达式为：

$$
I_L = g_L (E_L - V)
$$

其中：
- $g_L$ 为漏电通道电导（通常为常数）
- $E_L$ 为漏电反转电位
- $V$ 为膜电位

实现代码也相当简单：

```python
class IL(LeakageChannel):
    def __init__(
        self,
        size,
        g_max=0.1 * (u.mS / u.cm ** 2),
        E=-70. * u.mV,
        name=None,
    ):
        super().__init__(size=size, name=name)
        self.E = brainstate.init.param(E, self.varshape, allow_none=False)
        self.g_max = brainstate.init.param(g_max, self.varshape, allow_none=False)

    def current(self, V):
        return self.g_max * (self.E - V)
```

这个 `IL` 类提供了一个最简单的通道模型，它不涉及任何内部状态或动力学，是一个静态电导模型，常作为神经元模型中的基础部分。